In [1]:
import re

In [2]:
class SoundRegister():
    def __init__(self, name, instructions):
        self.instruction_re = re.compile('^([a-z]{3}) ([-0-9a-z]+)(?: ([-0-9a-z]+))?')
        self.last_played_sound = None
        self.curr_sound = None
        self.registers = dict()
        self.instruction_position = 0
        self.rcv_queue = []
        self.wait_register = None
        self.partner=None
        self.sends = 0

        self.name = name
        self.instructions = instructions
        
    def set_partner(self, partner):
        self.partner = partner
        
    def get_value(self, rhs):
        try:
            return int(rhs)
        except ValueError:
            return self.registers[rhs] if rhs in self.registers else 0
        
    def set_register(self, lhs, rhs):
        self.registers[lhs] = rhs

    def st(self, lhs, rhs):
        self.instruction_position += 1
        self.registers[lhs] = self.get_value(rhs)
        
    def add(self, lhs, rhs):
        self.instruction_position += 1
        if lhs not in self.registers:
            self.registers[lhs] = 0
        self.registers[lhs] += self.get_value(rhs)
        
    def mod(self, lhs, rhs):
        self.instruction_position += 1
        if lhs not in self.registers:
            self.registers[lhs] = 0
        self.registers[lhs] %= self.get_value(rhs)

    def mul(self, lhs, rhs):
        self.instruction_position += 1
        if lhs not in self.registers:
            self.registers[lhs] = 0
        self.registers[lhs] *= self.get_value(rhs)
        
#     def snd(self, freq):
#         self.instruction_position += 1
#         self.curr_sound = self.get_value(freq)
#         self.play_sound('snd', self.curr_sound)
        
#     def rcv(self, rhs):
#         self.instruction_position += 1
#         if not self.get_value(rhs):
#             return
#         self.play_sound('rcv', self.curr_sound)
        
    def jgz(self, lhs, rhs):
        if self.get_value(lhs):
            self.instruction_position += self.get_value(rhs)
        else:
            self.instruction_position += 1
            
    def snd(self, rhs):
        self.sends += 1
        print('{} sent {}'.format(self.name, self.sends))
        self.instruction_position += 1
        self.partner.message(self.get_value(rhs))
        
    def rcv(self, lhs=None):
        if not lhs and not self.wait_register:
            raise ValueError('Message sent to terminated SoundRegister {}'.format(self.name))
            
        if self.rcv_queue:
            register = lhs or self.wait_register
            self.registers[register] = self.rcv_queue.pop()
            self.instruction_position += 1
            self.wait_register = None
        else:
            self.wait_register = lhs
            
    def play_sound(self, cmd, sound):
        if (cmd == 'rcv'):
            print('RCV playing: ', sound)

    def message(self, value):
        self.rcv_queue.insert(0, value)
        if self.wait_register:
            self.rcv(self)
            self.play()

    def play(self):
        instruction_map = {
            'set': (self.st, 2),
            'add': (self.add, 2),
            'mod': (self.mod, 2),
            'mul': (self.mul, 2),
            'snd': (self.snd, 1),
            'rcv': (self.rcv, 1),
            'jgz': (self.jgz, 2),
        }

        while self.instruction_position >= 0 and self.instruction_position < len(self.instructions):
            instruction = self.instructions[self.instruction_position]
            #print('{} running instruction: {}'.format(self.name, instruction))
            m = self.instruction_re.match(instruction)
            if not m:
                raise ValueError('Invalid instruction {}'.format(instruction))
            if m[1] not in instruction_map:
                print('Command: ', m[1])
                raise ValueError('Invalid command {}'.format(instruction))
            cmd, args = instruction_map[m[1]]
            if args == 2:
                cmd(m[2], m[3])
            else:
                cmd(m[2])
            if self.wait_register:
                break

In [3]:
test_instructions = [
    'snd 1',
    'snd 2',
    'snd p',
    'rcv a',
    'rcv b',
    'rcv c',
    'rcv d',
    'set a 10',
]

In [5]:
with open('sound.txt') as fh:
    instructions = fh.readlines()
instructions = [i.strip() for i in instructions]

In [6]:
instructions

['set i 31',
 'set a 1',
 'mul p 17',
 'jgz p p',
 'mul a 2',
 'add i -1',
 'jgz i -2',
 'add a -1',
 'set i 127',
 'set p 316',
 'mul p 8505',
 'mod p a',
 'mul p 129749',
 'add p 12345',
 'mod p a',
 'set b p',
 'mod b 10000',
 'snd b',
 'add i -1',
 'jgz i -9',
 'jgz a 3',
 'rcv b',
 'jgz b -1',
 'set f 0',
 'set i 126',
 'rcv a',
 'rcv b',
 'set p a',
 'mul p -1',
 'add p b',
 'jgz p 4',
 'snd a',
 'set a b',
 'jgz 1 3',
 'snd b',
 'set f 1',
 'add i -1',
 'jgz i -11',
 'snd a',
 'jgz f -16',
 'jgz a -19']

In [4]:
s0 = SoundRegister('s0', test_instructions)
s1 = SoundRegister('s1', test_instructions)
s0.set_register('p', 0)
s1.set_register('p', 1)
s0.set_partner(s1)
s1.set_partner(s0)
s0.play()
s1.play()

s0 sent 1
s0 sent 2
s0 sent 3
s1 sent 1
s1 sent 2
s1 sent 3
